In [163]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#tutorial: https://www.tensorflow.org/tutorials/load_data/csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

In [164]:
path = "/content/drive/MyDrive/TELCO/telco_numeric.csv"
telco = pd.read_csv(path, index_col=0)

In [165]:
telco.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_Yes,InternetService_Fiber_optic,InternetService_No,OnlineSecurity_Yes,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,Contract_One_year,Contract_Two_year,PaperlessBilling_Yes,PaymentMethod_Credit_card_(automatic),PaymentMethod_Electronic_check,PaymentMethod_Mailed_check,Churn_Yes
0,0,1,29.85,29.85,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0
1,0,34,56.95,1889.50,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
2,0,2,53.85,108.15,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,1
3,0,45,42.30,1840.75,1,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0
4,0,2,70.70,151.65,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1


In [166]:
#multicollinear variables determined with inflation factors
telco = telco.drop(['MonthlyCharges', 'StreamingTV_Yes', 'PhoneService_Yes',
                          'TotalCharges', 'InternetService_No'], axis=1)

In [167]:
features = telco.copy()
labels = features.pop('Churn_Yes')
#features = StandardScaler().fit_transform(features) #this can be done with the model itself
features = np.array(features)

In [168]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=.2, stratify=labels)

In [169]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5634, 18), (1409, 18), (5634,), (1409,))

In [223]:
n_features = X_train.shape[1]
n_classes = 2
# define the keras model
#https://machinelearningmastery.com/neural-network-models-for-combined-classification-and-regression/
model = Sequential()
model.add(Dense(20, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
#model.add(Dense(1, activation='linear')) #linear regression
model.add(Dense(n_classes, activation='softmax'))

In [224]:
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [225]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
177/177 - 1s - loss: 0.5986 - accuracy: 0.7370
Epoch 2/10
177/177 - 0s - loss: 0.4608 - accuracy: 0.7813
Epoch 3/10
177/177 - 0s - loss: 0.4401 - accuracy: 0.7916
Epoch 4/10
177/177 - 0s - loss: 0.4285 - accuracy: 0.7982
Epoch 5/10
177/177 - 0s - loss: 0.4348 - accuracy: 0.7922
Epoch 6/10
177/177 - 0s - loss: 0.4241 - accuracy: 0.8000
Epoch 7/10
177/177 - 0s - loss: 0.4265 - accuracy: 0.8009
Epoch 8/10
177/177 - 0s - loss: 0.4268 - accuracy: 0.8005
Epoch 9/10
177/177 - 0s - loss: 0.4204 - accuracy: 0.8014
Epoch 10/10
177/177 - 0s - loss: 0.4215 - accuracy: 0.8072


In [226]:
# evaluate on test set
yhat = model.predict(X_test)
#error = mean_absolute_error(y_test, yhat) #linear reg
#print('MAE: %.3f' % error) #linear reg
# evaluate on test set
yhat = np.argmax(yhat, axis=-1).astype('int')
acc = accuracy_score(y_test, yhat)
print('Accuracy: %.3f' % acc)

Accuracy: 0.806


In [159]:
model.evaluate(X_test, y_test)

45/45 [==============================] - 0s 1ms/step - loss: 0.4092 - accuracy: 0.8197


[0.40923088788986206, 0.819730281829834]

In [162]:
yhat[:12] #it's better than guessing 0 every time
#but thre's not enough data to outperform traditional machine learning algorthms

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [161]:
1 - sum(y_test)/len(y_test)

0.7345635202271115